## Download all "Revue Esprit" archives


### How to
You need  a valid Account to access to the Archives.
With this account, we can retrieve your cookie session with Chrome Debugger.
Copy this cookie in the `cookie` variable below

In [ ]:
import os
import pandas as pd
import requests
import time
import pathlib
from tqdm.notebook import tqdm
from lxml import html
from http.cookies import BaseCookie

In [ ]:
cookie = "_ga=GA1.2.2050623063.1579637042; cookieconsent_status=dismiss; _gid=GA1.2.1618495606.1579957996; PHPSESSID=h9vdig327mhtqbr37emj37ema6; language=%84%EA%F8%5C%9AY%19%C0O%40V%97N%8E%C3y%D1%F3%FF%21%FB%B5%3C%9E%26%87_%19zN%E4%B5%3E7%B7%F5%C3%C3%22%83%07%11%DF%DD%82T%24%EE%E3%83m%1A%CC%15%0F%F1F%03%BD%12f%FB%DF%40; user=k%BD-vj%24%95%26%C0%F6d%88%FA%89%A0%2C%22%8D%2F%1C%BC%85%ED%18%87%15%F0%1D%B3%95%CC%12%E9W%3D%1C%FA%D4%BF%9E%F7J%F0%C4%81u%27b%0Bz%13w-%9F7%A8%2A%84%BD%17%E4%14%AE%28%86w%EA%A7%9C%94%C8%1B%1D%BD%07%D9T%E4%F7%D4Y%CC%EB%EB%A8%0A%7F%29%0E%B4%A7%A3%EFL%F5%17%A9%8Fl%B0kR%F8%3E%2B%87%22yt%0A%EA%C5%A5B%CC%D5%0E%FF%E0pM%E9%7E%ED9%8A%5D%F0%8A%19%C6%12%95%0E%A0%D5c%D5%40sC%CF%FD%E7%0B%1E%E6%28%18%ED%B4%F68%1A%98%02g%F9%12%0Bm%9B%1C%BBG1%D8%FA%60%F2%E3o%3B%06%5CA%A7Z%8D%C3%8A%DB%0BD%E3%28%00%98%CBiM%C6"

In [ ]:
pathlib.Path('./epub').mkdir(parents=True, exist_ok=True)
pathlib.Path('./pdf').mkdir(parents=True, exist_ok=True)
pathlib.Path('./data/pdf').mkdir(parents=True, exist_ok=True)
pathlib.Path('./data/epub').mkdir(parents=True, exist_ok=True)

In [ ]:
all_numbers = range(1,886)


def set_url_pdf(number):
    return f"https://esprit.presse.fr/download/numero/{number}/pdf"

def set_url_epub(number):
    return f"https://esprit.presse.fr/download/numero/{number}/epub" 



In [ ]:
cook = {"Host": "esprit.presse.fr",
"Connection": "keep-alive",
"Upgrade-Insecure-Requests": "1",
"DNT": "1",
"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36",
"Sec-Fetch-User": "?1",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"Sec-Fetch-Site": "same-origin",
"Sec-Fetch-Mode": "navigate",
"Referer": "https://esprit.presse.fr/tous-les-numeros/le-partage-de-l-universel/885",
"Accept-Encoding": "gzip, deflate, br",
"Accept-Language": "en-US,en;q=0.9,de;q=0.8,fr;q=0.7",
"Cookie":cookie
       }


### Download archives

In [ ]:
def download_pdf(number):
    url = set_url_pdf(number)
    r = requests.get(url, headers = cook)
    with open(f'.pdf/esprit_number_{number}.pdf', 'wb') as f:
        f.write(r.content)
        time.sleep(2)
        print(number)

def download_epub(number):
    url = set_url_epub(number)
    r = requests.get(url, headers = cook)
    with open(f'.epub/esprit_number_{number}.epub', 'wb') as f:
        f.write(r.content)
        time.sleep(2)
        print(number)
        
def download_all():
    for n in all_numbers:
        try:
            download_epub(n)
            download_pdf(n)
        except Exception as e:
            print(e)
            print(f'for number {n}')

### Retrieve metadata

In [ ]:
def get_metadata_old(number):
    url = f'https://esprit.presse.fr/tous-les-numeros/{number}'
    pageContent=requests.get(url)
    tree = html.fromstring(pageContent.content)
    
    title = tree.xpath('//h1[@class="title"]')
    title = title[0].text
    date = tree.xpath('//div[@class="date margin-top-30 margin-sm-top-10"]')
    date = date[0].text
    
    articles =tree.xpath('//h2[@class="h5 margin-0"]')
    articles = [a.text for a in articles]
    authors = tree.xpath('//p[@class="author-surtitle margin-0"]')
    authors = [a.text for a in authors]
    df = pd.DataFrame({'number':number,'date':date,'title':title,'article':articles,'authors':authors})
    return df

def get_metadata_new(number):
    url = f'https://esprit.presse.fr/tous-les-numeros/{number}'
    pageContent=requests.get(url)
    tree = html.fromstring(pageContent.content)
    title = tree.xpath('//h1[@class="title"]')
    title = title[0].text
    date = tree.xpath('//div[@class="date margin-top-30 margin-sm-top-10"]')
    date = date[0].text
    articles =tree.xpath('//*[self::h3 or self::h4][@class="h5 margin-0"]')
    articles = [a.text for a in articles]
    authors = tree.xpath('//p[@class="author-surtitle margin-0"]')
    authors = [a.text for a in authors]
    df = pd.DataFrame({'number':number,'date':date,'title':title,'article':articles,'authors':authors})
    return df


In [ ]:
def get_all_metadata():
    df_all= pd.DataFrame()
    
    for n in tqdm(all_numbers):
        try:
            df = get_metadata_old(n)      
        except Exception as e:          
            try:
                df = get_metadata_new(n)
            except Exception as e:
                print(e)
                print(f'for number:{n}')
        finally:
            df_all = pd.concat([df_all,df],axis=0)
    return df_all   

In [ ]:
df_all = get_all_metadata()

In [ ]:
df_all.to_csv('all_Esprits_articles.csv',encoding='utf-8',index=False)

### Renames all files

In [ ]:
df_metada = pd.read_csv('all_Esprits_articles.csv')
metadata_dic = df_all.drop_duplicates('number').set_index('number').T.to_dict()

In [ ]:
all_pdf = pathlib.Path('./pdf').glob('**/*')
files = [x for x in all_pdf if x.is_file()]


def rename_files(number,extension='pdf'):
    
    old_file = os.path.join(extension, f"esprit_number_{str(number)}.{extension}")
    print(old_file)
    if metadata_dic[number]['title'] is None:
        title=''
    else :
        title =  metadata_dic[number]['title']
    new_file_name = metadata_dic[number]['date'].replace('/','_')+'_'+title+'.'+extension
    new_file = os.path.join("./data/epub", new_file_name)
   
    os.rename(old_file, new_file)
    
    

In [ ]:
for n in all_numbers:
    try:
        rename_files(n,'pdf')
        rename_files(n,'epub')
    except Exception as e:
        print(e)
        print(n)

In [ ]:

df_metada[~df_metada.authors.isnull() &df_metada.authors.str.contains('Bourg')]